In [ ]:
# Pkg.add("ScikitLearn")
using ScikitLearn

In [ ]:
# Pkg.add("DecisionTree")
using DecisionTree

In [ ]:
# read in data from file
tmp = readdlm("listings_clean.csv", ',',header=true)
listings = tmp[1]
colnames = tmp[2]
size(listings)

In [ ]:
print(colnames)

In [ ]:
# training/test split
N_train = 2000
training_mask  = zeros(Bool,size(listings)[1])
srand(4)
training_indices = randperm(size(listings)[1])[1:N_train]
training_mask[training_indices] = true
testing_mask = .!training_mask;

In [ ]:
# save training data to csv
writedlm("listings_clean_train.csv",listings[training_mask,:],',')

In [ ]:
# save testing data to csv
writedlm("listings_clean_test.csv",listings[testing_mask,:],',')

In [ ]:
# using lasso regression, we achieved an out of sample R^2 of 0.53
# let's try with random forest
n_features = size(listings)[2] - 1
model = RandomForestRegressor(nsubfeatures=round(Int,n_features/3),ntrees=500,maxlabels=1)
fit!(model, listings[training_mask,2:end],listings[training_mask,1])

In [ ]:
# out of sample performance
y_test = listings[testing_mask,1]
y_pred = predict(model,listings[testing_mask,2:end])
1 - sum((y_test - y_pred).^2) / sum((y_test - mean(y_test)) .^ 2)

In [ ]:
# let's try with just a single decision tree
model = DecisionTreeRegressor(pruning_purity_threshold = 0.8,maxlabels = 100)
fit!(model, listings[training_mask,2:end],listings[training_mask,1])

In [ ]:
y_pred = predict(model,listings[testing_mask,2:end])
1 - sum((y_test - y_pred).^2) / sum((y_test - mean(y_test)).^2)

In [ ]:
# serialize model (i.e. save to disk in a binary format)
using JLD
JLD.save("model.jld","model",model)

In [ ]:
model = JLD.load("model.jld","model")